In [1]:
import numpy as np
import sys
from scipy.special import erfcinv as erfcinv
import tqdm as tqdm
import time
import scipy.io as sio
import lax_wendroff as lw
import ipywidgets as widgets

In [2]:
# SECTION 0: Definitions (normally don't modify this section)

# Possible initial conditions of the height field
# only allow Gaussian blob, but make slider so can adjust height

In [3]:
# Possible orographies
FLAT=0;
SLOPE=1;
GAUSSIAN_MOUNTAIN=2;
EARTH_OROGRAPHY=3;
SEA_MOUNT=4;
SEA_MOUNT2=5;
oro = widgets.Dropdown(
    options=[('FLAT',0),('SLOPE', 1), ('GAUSSIAN_MOUNTAIN', 2), ('EARTH_OROGRAPHY', 3), ('SEA_MOUNT', 4), ('SEA_MOUNT2', 5)],
    value=1,
    description='Orography:',
)

In [4]:
container2 = widgets.HBox([oro])
container2

In [5]:
# SECTION 1: Configuration
g    = 9.81;                # Acceleration due to gravity (m/s2)
f=0.;
beta=0.;

dt_mins              = 3.;   # Timestep (minutes)
output_interval_mins = 60.;  # Time between outputs (minutes)
forecast_length_days = 12.0;   # Total simulation length (days)

initial_conditions = 4

# If you change the number of gridpoints then orography=EARTH_OROGRAPHY
# or initial_conditions=REANALYSIS won't work
nx=254; # Number of zonal gridpoints
ny=50;  # Number of meridional gridpoints
#ny=3;

dx=100.0e3; # Zonal grid spacing (m)
dy=dx;      # Meridional grid spacing

# Specify the range of heights to plot in metres
plot_height_range = np.array([9500., 10500.]);

In [6]:
# SECTION 2: Act on the configuration information
dt = dt_mins*60.0; # Timestep (s)
output_interval = output_interval_mins*60.0; # Time between outputs (s)
forecast_length = forecast_length_days*24.0*3600.0; # Forecast length (s)
nt = int(np.fix(forecast_length/dt)+1); # Number of timesteps
timesteps_between_outputs = np.fix(output_interval/dt);
noutput = int(np.ceil(nt/timesteps_between_outputs)); # Number of output frames

x=np.mgrid[0:nx]*dx; # Zonal distance coordinate (m)
y=np.mgrid[0:ny]*dy; # Meridional distance coordinate (m)
[Y,X] = np.meshgrid(y,x); # Create matrices of the coordinate variables

In [7]:
steep = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=12.0,
    step=1.0,
    description='Pente:',
    continuous_update=False
)
steep

IntSlider(value=1, continuous_update=False, description='Pente:', max=12, min=1)

In [8]:
orography = oro.value
def run_model(orography):
    # Create the orography field "H"
    if orography == FLAT:
       H = np.zeros((nx, ny));
    elif orography == SLOPE:
       steepness = 4000.*steep.value/12.
       H = steepness*2.*np.abs((np.mean(x)-X)/np.max(x));
    elif orography == GAUSSIAN_MOUNTAIN:
       std_mountain_x = 5.*dx; # Std. dev. of mountain in x direction (m)
       std_mountain_y = 5.*dy; # Std. dev. of mountain in y direction (m)
       H = 4000.*np.exp(-0.5*((X-np.mean(x))/std_mountain_x)**2. \
                  -0.5*((Y-np.mean(y))/std_mountain_y)**2.);
    elif orography == SEA_MOUNT:
       std_mountain = 40.0*dy; # Standard deviation of mountain (m)
       H = 9250.*np.exp(-((X-np.mean(x))**2.+(Y-0.5*np.mean(y))**2.)/(2.*std_mountain**2.));
    elif orography == SEA_MOUNT2:
       std_mountain = 40.0*dy; # Standard deviation of mountain (m)
       H = 9550.*np.exp(-((X-np.mean(x))**2.+(Y-0.5*np.mean(y))**2.)/(2.*std_mountain**2.));
       H = np.concatenate((H[len(x)//2:,:],H[:len(x)//2,:]),axis=0)
    elif orography == EARTH_OROGRAPHY:
       mat_contents = sio.loadmat('digital_elevation_map.mat')
       H = mat_contents['elevation'];
       # Enforce periodic boundary conditions in x
       H[[0, -1],:]=H[[-2, 1],:];
    else:
       print('Don''t know what to do with orography=' + np.num2str(orography)); 
       sys.exit()
        
    # Create the initial height field 
    std_blob = 8.0*dy; # Standard deviation of blob (m)
    #height = 9750. + 1000.*np.exp(-((X-0.25*np.mean(x))**2.+(Y-np.mean(y))**2.)/(2.* \
    #                                                     std_blob**2.));
    height = 9750. + 1000.*np.exp(-((X-1.1*np.mean(x))**2.+(Y-np.mean(y))**2.)/(2.* \
                                                     std_blob**2.));
    # Coriolis parameter as a matrix of values varying in y only
    F = f+beta*(Y-np.mean(y));
    
    # Initialize the wind to rest
    u=np.zeros((nx, ny));
    v=np.zeros((nx, ny));
    
    # Define h as the depth of the fluid (whereas "height" is the height of
    # the upper surface)
    h = height - H;
    
    # Initialize the 3D arrays where the output data will be stored
    u_save = np.zeros((nx, ny, noutput));
    v_save = np.zeros((nx, ny, noutput));
    h_save = np.zeros((nx, ny, noutput));
    t_save = np.zeros((noutput,1));
    
    # Index to stored data
    i_save = 0;
    # SECTION 3: Main loop
    #print('Running model: orography is ',orography)
    for n in range(0,nt):
       # Every fixed number of timesteps we store the fields
       if np.mod(n,timesteps_between_outputs) == 0:
        max_u = np.sqrt(np.max(u[:]*u[:]+v[:]*v[:]));
      
        #print("Time = %f hours (max %f); max(|u|) = %f"  
        #  % ((n)*dt/3600., forecast_length_days*24., max_u) )
 
        u_save[:,:,i_save] = u;
        v_save[:,:,i_save] = v;
        h_save[:,:,i_save] = h;
        t_save[i_save] = (n)*dt;
        i_save = i_save+1;
        

        # Compute the accelerations
        u_accel = F[1:-1,1:-1]*v[1:-1,1:-1] \
              - (g/(2.*dx))*(H[2:,1:-1]-H[0:-2,1:-1]);
        v_accel = -F[1:-1,1:-1]*u[1:-1,1:-1] \
              - (g/(2.*dy))*(H[1:-1,2:]-H[1:-1,0:-2]);

        # Call the Lax-Wendroff scheme to move forward one timestep
        (unew, vnew, h_new) = lw.lax_wendroff(dx, dy, dt, g, u, v, h, u_accel, v_accel);

        # Update the wind and height fields, taking care to enforce 
        # boundary conditions   
        u[1:-1,1:-1] = unew;
        v[1:-1,1:-1] = vnew;
        
        # first x-slice
        u[0,1:-1]=unew[-1,:]
        u[0,0]=unew[-1,0]
        u[0,-1]=unew[-1,-1]
        v[0,1:-1]=vnew[-1,:]
        v[0,0]=vnew[-1,0]
        v[0,-1]=vnew[-1,-1]
        # last x-slice
        u[-1,1:-1]=unew[0,:]
        u[-1,0]=unew[0,0]
        u[-1,-1]=unew[0,-1]
        v[-1,1:-1]=vnew[0,:]
        v[-1,0]=vnew[0,0]
        v[-1,-1]=vnew[0,-1]
   
        # no flux from north / south
        v[:,[0,-1]]=0.;
        # interior
        h[1:-1,1:-1] = h_new;
        # first x-slice
        h[0,1:-1]=h_new[-1,:]
        # last x-slice
        h[-1,1:-1]=h_new[0,:]
    return H, height, t_save, h_save

In [9]:
#from IPython.core.display import HTML
#HTML('''<script> </script> <form action="javascript:IPython.notebook.execute_cells_below()"><input type="submit" id="toggleButton" value="Refresh"></form>''')


In [46]:


import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display

import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.animation as animation
from scipy.ndimage import gaussian_filter1d

%matplotlib widget
#%matplotlib notebook

button = widgets.Button(description='Refresh')
display(button)

fig, (ax,ax1) = plt.subplots(2,1)
ax.set_xlabel('Distance')
ax.set_ylabel('Niveau de la mer [m]')

ax1.set_xlabel('Distance')
ax1.set_ylabel('Topographie [m]')
plt.subplots_adjust(hspace=0.35)

#plt.close()
def get_fig():
    ax.plot(np.arange(10), np.random.random(10))

#get_fig()

def update_line(i, plotslice, t_save, hmax, endtime):
    ax.cla()
    line, = ax.plot(plotslice[:-5,i],'b-')
    ax.set_ylim(9500,10000)
    title = ax.text(0, 10005, 'Niveau moyen = 9750 m');
    ax.set_xlabel('Distance')
    ax.set_ylabel('Niveau de la mer [m]')
    if i == endtime-1:
        t2 = ax.text(20, 9550, 'Hauteur de vague maximale = %.1f m' % [np.max(plotslice[-5,:])-9750.][0], fontsize=14, color='r')

def get_anim(fig,H,t_save,h_save):
    container = []
    
    mid = np.where((x - np.mean(x)) > 0)[0]
    
    plotslice = H[mid,25][:,None]+h_save[mid,25,:]
    # decide when to stop plotting
    targetpoint = plotslice[-5,:]
    hmax = np.max(plotslice[-5,:] - H[mid[-5],25])
    smooth = gaussian_filter1d(targetpoint, 2)
    smooth_d2 = np.gradient(smooth)
    infls = np.where(np.diff(np.sign(smooth_d2)))[0]
    if len(infls):
        maxtime = np.where(targetpoint == np.nanmax(targetpoint))[0]
        if len(infls[infls > maxtime]):
            endtime = infls[infls > maxtime][0]
        else:
            endtime = len(targetpoint)
    else:
        endtime = len(targetpoint)
    ax1.plot(H[np.size(H,0)//2:-5,25],'k-')
    ax1.set_xlabel('Distance')
    ax1.set_ylabel('Topographie [m]')
    #widgets.interact(update_line, i=widgets.Play(min=0, max=endtime-1, step=1, interval=75), plotslice = widgets.fixed(plotslice), t_save = widgets.fixed(t_save), hmax = widgets.fixed(hmax), endtime = widgets.fixed(endtime))
    test = widgets.interact(update_line, i=(0, endtime-1, 1), plotslice = widgets.fixed(plotslice), t_save = widgets.fixed(t_save), hmax = widgets.fixed(hmax), endtime = widgets.fixed(endtime))
    

def on_button_clicked(b):
    plt.cla()
    H, height, t_save, h_save = run_model(oro.value)
    get_anim(fig,H,t_save,h_save)
    
button.on_click(on_button_clicked)



Button(description='Refresh', style=ButtonStyle())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=144, description='i', max=288), Output()), _dom_classes=('widget-interac…

98.86472082184628

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(20, 9550, 'Hauteur de vague maximale = 98.9')

(254, 50)